Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [2]:
%run -i "assignment_helper.py"

Available frameworks:
Forest SDK
Qiskit
D-Wave Ocean


# State preparation

Preparing a state in a particular encoding can, in itself, give rise to interesting kernels. This is what talked about in the lecture on the interference kernel, and Maria Schuld's guest lecture expands on the idea. Let us work with an easy data set of two vectors, $S = \{(\begin{bmatrix}0 \\ 1\end{bmatrix}, 0), (\begin{bmatrix}\sqrt{2}/2 \\ \sqrt{2}/2\end{bmatrix}, 1)\}$. Let's have a test instance $\begin{bmatrix}1 \\ 0\end{bmatrix}$. We will build the interference circuit for this.

**Exercise 1** (1 point). Create a circuit in your preferred framework that works on four qubits: ancilla, index, data, and class. Put the ancilla and index qubits into a uniform superposition. Prepare and entangle the test instance with the ground state of the ancilla. Put an identity gate on the class qubit. Place your solution in an object called `circuit`. 

In [3]:
ancilla_qubit = 0
index_qubit = 1
data_qubit = 2
class_qubit = 3
###
### YOUR CODE HERE
###
from pyquil import Program, get_qc
from pyquil.gates import *

training_set = [[0, 1], [1/np.sqrt(2), 1/np.sqrt(2)]]
labels = [0, 1]
test_set = [1, 0]

circuit = Program()
circuit += H(ancilla_qubit)
circuit += H(index_qubit)

circuit += X(ancilla_qubit)
circuit += CNOT(ancilla_qubit, data_qubit)
circuit += X(ancilla_qubit)

circuit += I(class_qubit)

In [4]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0.5+0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]))

**Exercise 2** (1 point). Extend the circuit to prepare the first training instance and entanle it with the excited state of the ancilla and ground state of the index qubit.

In [5]:
###
### YOUR CODE HERE
###
circuit += X(index_qubit)
circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += X(index_qubit)

In [6]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]))

**Exercise 3** (2 points). Extend the circuit to prepare the second training instance and entangle it with the excited state of the ancilla and the excited of the index qubit.

In [7]:
###
### YOUR CODE HERE [000 1/2 1/2 1/2 1/2 000000000]
###
circuit += X(ancilla_qubit)
circuit += H(data_qubit)
circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += H(data_qubit)
circuit += X(ancilla_qubit)
# [000 1/2 1/2 1/2 -1/2 000000000]

In [8]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([ 0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j,  0.5+0.j,  0.5+0.j,
       -0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j]))

**Exercise 4** (1 point). Finish the state preparation circuit by flipping the class qubit conditioned on the index qubit.

In [9]:
###
### YOUR CODE HERE
###
circuit += CNOT(index_qubit, class_qubit)

In [10]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j,  0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j, 0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j]))

# Interference as a kernel

**Exercise 5** (1 point). At this point, our state is $\frac{1}{2\sqrt{2}}\sum_{i=0}^1|0\rangle|i\rangle(|x_t\rangle+|x_i\rangle)|y_i\rangle+|1\rangle|i\rangle(|x_t\rangle-|x_i\rangle)|y_i\rangle$, where $|x_t\rangle$ is the encoded test instance and $|x_i\rangle$ is a training instance. Apply the Hadamard gate on the ancilla to apply the interference.

In [11]:
###
### YOUR CODE HERE
###
circuit += H(ancilla_qubit)

In [12]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([ 0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.70710678+0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,  0.35355339+0.j, -0.35355339+0.j,
        0.        +0.j,  0.        +0.j, -0.35355339+0.j, -0.35355339+0.j]))

If we measure the ancilla, the outcome probability of observing 0 will be $\frac{1}{4N}\sum_{i=1}^N |x_t + x_i|^2$. Performing post-selection on the 0 outcome, we can calculate the kernel and the probability of the test instance belonging to either class.